In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.svm import SVC
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import hamming_loss, accuracy_score, precision_score
import re

# Load dataset
aspects_df = pd.read_csv('semeval2014-1.csv')

# Display the first few rows of the dataset
print(aspects_df.head())

# Aspect labels
aspect_labels = ["service", "food", "anecdotes/miscellaneous", "price", "ambience"]

# Function to handle contractions and negations
def preprocess_text(text):
    # Define contractions
    contractions = {
        "can't": "cannot",
        "won't": "will not",
        "shan't": "shall not",
        "n't": " not",
        "'re": " are",
        "'s": " is",
        "'d": " would",
        "'ll": " will",
        "'t": " not",
        "'ve": " have",
        "'m": " am"
    }

    # Replace contractions
    for contraction, replacement in contractions.items():
        text = re.sub(contraction, replacement, text)

    # Handle negations
    negations = ["not", "never", "no", "none", "nobody", "nothing", "neither", "nowhere", "hardly", "scarcely", "barely", "cannot"]
    text_tokens = text.split()
    for i in range(len(text_tokens) - 1):
        if text_tokens[i] in negations:
            text_tokens[i + 1] = "NOT_" + text_tokens[i + 1]
    return " ".join(text_tokens)

# Apply preprocessing to text data
aspects_df["text"] = aspects_df["text"].apply(preprocess_text)

# Extract features and labels for aspect classification
X_aspect = aspects_df["text"]
y_aspect = np.asarray(aspects_df[aspects_df.columns[1:6]])

# Vectorize the text data for aspect classification
vectorizer_aspect = TfidfVectorizer(max_features=3000, max_df=0.85)
X_aspect_tfidf = vectorizer_aspect.fit_transform(X_aspect)

# Split the dataset into training and testing sets for aspect classification
X_train_aspect, X_test_aspect, y_train_aspect, y_test_aspect = train_test_split(X_aspect_tfidf, y_aspect, test_size=0.33, random_state=42)

# Initialize SVM classifier for aspect classification
svm_classifier_aspect = SVC(kernel='linear')  # You can specify different kernels if needed

# Wrap SVM classifier in MultiOutputClassifier for multi-label classification
multi_label_classifier_aspect = MultiOutputClassifier(svm_classifier_aspect, n_jobs=-1)

# Train the classifier for aspect classification
multi_label_classifier_aspect.fit(X_train_aspect, y_train_aspect)

# Extract features and labels for sentiment analysis
X_sentiment = aspects_df["text"]
y_sentiment = aspects_df["review"]

# Vectorize the text data for sentiment analysis using bag-of-words model
vectorizer_sentiment = CountVectorizer(max_features=3000, max_df=0.85)
X_sentiment_bow = vectorizer_sentiment.fit_transform(X_sentiment)

# Split the dataset into training and testing sets for sentiment analysis
X_train_sentiment, X_test_sentiment, y_train_sentiment, y_test_sentiment = train_test_split(X_sentiment_bow, y_sentiment, test_size=0.33, random_state=42)

# Initialize SVM classifier for sentiment analysis
svm_classifier_sentiment = SVC(kernel='linear')  # You can specify different kernels if needed

# Train the classifier for sentiment analysis
svm_classifier_sentiment.fit(X_train_sentiment, y_train_sentiment)

# Predict and evaluate on the test set for aspect classification
predicted_aspect = multi_label_classifier_aspect.predict(X_test_aspect)
print(f"Aspect Classification Accuracy: {accuracy_score(y_test_aspect, predicted_aspect)}")
print(f"Aspect Classification Hamming Loss: {hamming_loss(y_test_aspect, predicted_aspect)}")
print(f"Aspect Classification Precision: {precision_score(y_test_aspect, predicted_aspect, average='micro')}")

# Predict and evaluate on the test set for sentiment analysis
predicted_sentiment = svm_classifier_sentiment.predict(X_test_sentiment)
print(f"Sentiment Analysis Accuracy: {accuracy_score(y_test_sentiment, predicted_sentiment)}")
print(f"Sentiment Analysis Precision: {precision_score(y_test_sentiment, predicted_sentiment, average='micro', zero_division=1)}")

# Function to predict aspects and sentiment for new sentences
def predict_aspects_and_sentiment(new_sentences):
    # Apply preprocessing to new sentences
    new_sentences = [preprocess_text(sentence) for sentence in new_sentences]

    new_sentences_aspect_tfidf = vectorizer_aspect.transform(new_sentences)
    predicted_aspects = multi_label_classifier_aspect.predict(new_sentences_aspect_tfidf)
    
    new_sentences_sentiment_bow = vectorizer_sentiment.transform(new_sentences)
    predicted_sentiments = svm_classifier_sentiment.predict(new_sentences_sentiment_bow)
    
    # Convert numerical predictions to aspect labels
    predicted_aspect_labels = []
    for aspects in predicted_aspects:
        labels = [aspect_labels[i] for i, val in enumerate(aspects) if val == 1]
        predicted_aspect_labels.append("|".join(labels))
    
    return predicted_aspects, predicted_sentiments, predicted_aspect_labels

# Example usage
new_sentences = ["the service was too good"]
predicted_aspects, predicted_sentiments, predicted_aspect_labels = predict_aspects_and_sentiment(new_sentences)
print(f"Predicted Aspects: {predicted_aspects}")
print(f"Predicted Sentiments: {predicted_sentiments}")
print(f"Predicted Aspect Labels: {predicted_aspect_labels}")


                                                text  service  food  \
0                but the staff was so horrible to us        1     0   
1  to be completely fair the only redeeming facto...        0     1   
2  the food is uniformly exceptional with a very ...        0     1   
3  where gabriela personaly greets you and recomm...        1     0   
4  for those that go once and dont enjoy it all i...        0     0   

   anecdotes/miscellaneous  price  ambience    review  
0                        0      0         0  negative  
1                        1      0         0   neutral  
2                        0      0         0  positive  
3                        0      0         0  positive  
4                        1      0         0   neutral  
Aspect Classification Accuracy: 0.5916749256689792
Aspect Classification Hamming Loss: 0.11298315163528246
Aspect Classification Precision: 0.8443051201671892
Sentiment Analysis Accuracy: 0.6134786917740337
Sentiment Analysis Precision: 